# Module 07: Performance Optimization for Windows

**Difficulty**: ⭐⭐⭐  
**Estimated Time**: 60 minutes  
**Prerequisites**: 
- Module 02 (npm) OR Module 03 (Docker) completed
- n8n installed and running
- Basic understanding of system resources (RAM, CPU, disk)

## Learning Objectives

By the end of this notebook, you will be able to:

1. Optimize memory usage for npm and Docker installations
2. Configure WSL2 resource limits to prevent excessive RAM consumption
3. Understand file system performance differences and optimize storage
4. Implement battery-saving strategies for laptop deployments
5. Choose and configure appropriate databases for your workload
6. Set up execution data pruning to prevent database bloat

## Why Performance Optimization Matters

Windows laptops have unique performance characteristics compared to always-on servers:

- **Limited resources**: Laptops have constrained RAM, CPU, and battery
- **Shared workload**: n8n competes with browser, IDE, and other applications
- **Mobility**: Battery life matters more than raw performance
- **WSL2 overhead**: Docker on Windows adds virtualization layer

### Real-World Impact

**Without optimization**:
- Docker Desktop + WSL2 consumes 8-16GB RAM at idle
- Battery drains 30-50% faster
- Laptop fans run constantly
- System becomes sluggish
- Database grows to 10+ GB in months

**With optimization**:
- Memory usage capped at 4GB
- Battery impact reduced by 60%
- Quiet, cool operation
- Fast, responsive system
- Database stays under 500MB

### The Performance Triangle

```
     Performance
          ▲
         / \
        /   \
       /     \
      /       \
     /         \
    /___________\
Battery ◄──────► Resources
  Life            Used
```

You can optimize two of three: performance, battery life, or resource usage. Choose based on your use case.

## Part 1: Memory Management

### Understanding n8n Memory Usage

| Installation Method | Idle Memory | Light Load | Heavy Load | Peak |
|---------------------|-------------|------------|------------|------|
| npm (native) | 100-320 MB | 300-600 MB | 600MB-1.5GB | 2-4GB |
| Docker Desktop | 2-4 GB overhead + 100-320 MB | 2.5-4.5 GB | 3-6 GB | 6-12GB |
| WSL2 (without limits) | 1-2 GB | 2-8 GB | 4-16 GB | 16+ GB |

**Key insight**: Docker overhead is 20-40x higher than npm at idle.

### Memory Consumption Factors

1. **Workflow complexity**: More nodes = more memory
2. **Data size**: Processing large datasets increases RAM usage
3. **Execution history**: Stored execution data accumulates
4. **Binary data**: Images, files, PDFs consume significant memory
5. **Code nodes**: JavaScript execution requires memory for V8 engine

### npm Installation Memory Optimization

npm installations are already memory-efficient, but you can optimize further:

#### 1. Limit Node.js Memory

Set maximum heap size:

```batch
SET NODE_OPTIONS=--max-old-space-size=1024
n8n
```

This limits n8n to 1GB RAM (1024 MB). Adjust based on available memory:
- 4GB system RAM: 512-1024 MB
- 8GB system RAM: 1024-2048 MB
- 16GB+ system RAM: 2048-4096 MB

#### 2. Reduce Execution Data Storage

```batch
SET EXECUTIONS_DATA_SAVE_ON_SUCCESS=none
SET EXECUTIONS_DATA_SAVE_ON_ERROR=all
SET EXECUTIONS_DATA_SAVE_MANUAL_EXECUTIONS=false
```

This only saves execution data for errors, reducing memory and database size.

### Docker Installation Memory Optimization

Docker requires WSL2 configuration (covered in Part 2) plus container limits:

#### Docker Compose with Memory Limits

```yaml
services:
  n8n:
    image: docker.n8n.io/n8nio/n8n
    container_name: n8n
    ports:
      - "5678:5678"
    environment:
      - GENERIC_TIMEZONE=America/New_York
      - N8N_BASIC_AUTH_ACTIVE=true
      - N8N_BASIC_AUTH_USER=admin
      - N8N_BASIC_AUTH_PASSWORD=secure_password
    volumes:
      - n8n_data:/home/node/.n8n
    restart: unless-stopped
    deploy:
      resources:
        limits:
          memory: 1G
          cpus: '2.0'
        reservations:
          memory: 256M
          cpus: '0.5'

volumes:
  n8n_data:
```

**Note**: Requires Docker Compose v3.8+ and may need `docker compose --compatibility up -d`

In [ ]:
# Memory usage analyzer

import subprocess
import platform

def analyze_system_memory():
    """
    Analyze system memory and provide n8n optimization recommendations.
    """
    print("=" * 70)
    print("SYSTEM MEMORY ANALYSIS & N8N OPTIMIZATION")
    print("=" * 70)
    
    if platform.system() == 'Windows':
        try:
            # Get total memory using PowerShell
            result = subprocess.run(
                ['powershell', '-Command', 
                 '(Get-CimInstance Win32_OperatingSystem).TotalVisibleMemorySize'],
                capture_output=True,
                text=True,
                timeout=5
            )
            
            total_kb = int(result.stdout.strip())
            total_mb = total_kb / 1024
            total_gb = total_mb / 1024
            
            # Get free memory
            result = subprocess.run(
                ['powershell', '-Command',
                 '(Get-CimInstance Win32_OperatingSystem).FreePhysicalMemory'],
                capture_output=True,
                text=True,
                timeout=5
            )
            
            free_kb = int(result.stdout.strip())
            free_mb = free_kb / 1024
            free_gb = free_mb / 1024
            
            used_gb = total_gb - free_gb
            usage_percent = (used_gb / total_gb) * 100
            
            print(f"\n📊 CURRENT MEMORY STATUS")
            print("─" * 70)
            print(f"Total RAM: {total_gb:.2f} GB")
            print(f"Used RAM:  {used_gb:.2f} GB ({usage_percent:.1f}%)")
            print(f"Free RAM:  {free_gb:.2f} GB")
            
        except Exception as e:
            print(f"\n⚠️  Could not determine memory: {e}")
            print("Assuming 8GB for recommendations...")
            total_gb = 8
    else:
        print("\nℹ️  Running on non-Windows system")
        print("Assuming 8GB for recommendations...")
        total_gb = 8
    
    # Generate recommendations
    print("\n" + "─" * 70)
    print("💡 N8N MEMORY OPTIMIZATION RECOMMENDATIONS")
    print("─" * 70)
    
    if total_gb <= 4:
        print("\n⚠️  LOW MEMORY SYSTEM (4GB or less)")
        print("\nRecommended configuration:")
        print("  • Installation method: npm (NOT Docker)")
        print("  • Node.js max memory: 512 MB")
        print("    SET NODE_OPTIONS=--max-old-space-size=512")
        print("  • Save executions: errors only")
        print("  • Database: SQLite (PostgreSQL too heavy)")
        print("\n⚠️  Expect performance issues with complex workflows")
        
    elif total_gb <= 8:
        print("\n✅ MODERATE MEMORY SYSTEM (6-8GB)")
        print("\nFor npm installation:")
        print("  • Node.js max memory: 1024 MB")
        print("    SET NODE_OPTIONS=--max-old-space-size=1024")
        print("  • Save executions: errors + manual tests")
        print("  • Database: SQLite recommended")
        print("\nFor Docker installation:")
        print("  • WSL2 memory limit: 4 GB")
        print("  • Container memory limit: 1 GB")
        print("  • WARNING: Docker may cause system sluggishness")
        print("  • Consider npm installation instead")
        
    elif total_gb <= 16:
        print("\n✅ GOOD MEMORY SYSTEM (12-16GB)")
        print("\nFor npm installation:")
        print("  • Node.js max memory: 2048 MB")
        print("    SET NODE_OPTIONS=--max-old-space-size=2048")
        print("  • Save executions: all or errors only (your choice)")
        print("  • Database: SQLite or PostgreSQL")
        print("\nFor Docker installation:")
        print("  • WSL2 memory limit: 6 GB")
        print("  • Container memory limit: 2 GB")
        print("  • Docker is viable but npm still more efficient")
        
    else:
        print("\n✅ HIGH MEMORY SYSTEM (16GB+)")
        print("\nFor npm installation:")
        print("  • Node.js max memory: 4096 MB")
        print("    SET NODE_OPTIONS=--max-old-space-size=4096")
        print("  • Save executions: all recommended")
        print("  • Database: PostgreSQL recommended for production")
        print("\nFor Docker installation:")
        print("  • WSL2 memory limit: 8 GB")
        print("  • Container memory limit: 4 GB")
        print("  • Docker is fully viable")
    
    print("\n" + "=" * 70)
    print("\n📝 GENERAL TIPS:")
    print("  • Close unnecessary applications when running n8n")
    print("  • Enable execution data pruning")
    print("  • Store binary data in filesystem, not database")
    print("  • Monitor memory usage with Task Manager")
    print("=" * 70)

# Run the analysis
analyze_system_memory()

## Part 2: WSL2 Configuration (.wslconfig)

**Applies to**: Docker Desktop users (and anyone using WSL2)

### The WSL2 Memory Problem

By default, WSL2 can consume up to 50% of total system RAM and rarely releases it:

- On 16GB system: WSL2 takes up to 8GB
- On 32GB system: WSL2 takes up to 16GB
- Memory persists even when Docker is idle
- `vmmem` process shows high usage in Task Manager

### The .wslconfig Solution

Create or edit `.wslconfig` in your user profile directory:

**Location**: `C:\Users\YourUsername\.wslconfig`

### Recommended Configurations

#### For 8GB System

```ini
[wsl2]
# Limit WSL2 to 4GB RAM (50% of total)
memory=4GB

# Use 2 CPU cores
processors=2

# Limit swap file to 2GB
swap=2GB

# Swap file location (optional)
# swapFile=C:\temp\wsl-swap.vhdx

# Enable localhost forwarding
localhostForwarding=true
```

#### For 16GB System

```ini
[wsl2]
# Limit WSL2 to 6GB RAM (preserves 10GB for Windows)
memory=6GB

# Use 4 CPU cores
processors=4

# Limit swap to 4GB
swap=4GB

# Enable localhost forwarding
localhostForwarding=true
```

#### For 32GB System

```ini
[wsl2]
# Limit WSL2 to 8GB RAM
memory=8GB

# Use 6 CPU cores
processors=6

# Limit swap to 8GB
swap=8GB

# Enable localhost forwarding
localhostForwarding=true
```

### Apply Configuration

After creating/editing `.wslconfig`:

```powershell
# Shut down all WSL2 instances
wsl --shutdown

# Wait 8-10 seconds for complete shutdown

# Start Docker Desktop (or WSL2)
# Settings will be applied automatically
```

### Verify Configuration

```powershell
# Check WSL2 memory usage in Task Manager
# Look for "Vmmem" process

# Or use PowerShell
Get-Process vmmem
```

### Advanced Options

```ini
[wsl2]
memory=6GB
processors=4
swap=4GB
localhostForwarding=true

# These settings are more advanced:

# Swap file location (place on SSD for better performance)
swapFile=D:\wsl-swap.vhdx

# Virtual hard disk size (default: 256GB)
# Only increase if you need more disk space
# guiApplications=false  # Disable WSL GUI apps to save resources

# Kernel parameters (advanced users only)
# kernelCommandLine = vsyscall=emulate
```

In [ ]:
# .wslconfig generator

import os
from pathlib import Path

def generate_wslconfig(total_ram_gb):
    """
    Generate optimized .wslconfig based on system RAM.
    
    Args:
        total_ram_gb: Total system RAM in GB
    
    Returns:
        .wslconfig file contents as string
    """
    print("=" * 70)
    print("WSL2 CONFIGURATION GENERATOR")
    print("=" * 70)
    
    # Calculate optimal settings
    if total_ram_gb <= 4:
        wsl_memory = "3GB"
        processors = 2
        swap = "2GB"
        recommendation = "⚠️  MINIMAL - Consider npm instead of Docker"
    elif total_ram_gb <= 8:
        wsl_memory = "4GB"
        processors = 2
        swap = "2GB"
        recommendation = "✅ ADEQUATE for light Docker usage"
    elif total_ram_gb <= 16:
        wsl_memory = "6GB"
        processors = 4
        swap = "4GB"
        recommendation = "✅ GOOD for Docker Desktop"
    else:
        wsl_memory = "8GB"
        processors = 6
        swap = "8GB"
        recommendation = "✅ EXCELLENT for Docker Desktop"
    
    # Generate config file content
    config_content = f"""[wsl2]
# WSL2 Configuration for {total_ram_gb}GB System
# Generated by n8n Windows Automation Course

# Memory limit (prevents WSL2 from consuming all RAM)
memory={wsl_memory}

# CPU cores allocated to WSL2
processors={processors}

# Swap file size (virtual memory)
swap={swap}

# Enable localhost forwarding (required for n8n access)
localhostForwarding=true

# Optional: Specify swap file location
# swapFile=C:\\temp\\wsl-swap.vhdx

# Optional: Disable GUI apps to save resources
# guiApplications=false
"""
    
    print(f"\n📊 System RAM: {total_ram_gb} GB")
    print(f"📈 Recommendation: {recommendation}")
    print("\n" + "─" * 70)
    print("GENERATED CONFIGURATION:")
    print("─" * 70)
    print(config_content)
    print("─" * 70)
    
    # Get user profile path
    user_profile = os.environ.get('USERPROFILE', 'C:\\Users\\YourUsername')
    config_path = Path(user_profile) / '.wslconfig'
    
    print("\n📝 INSTALLATION INSTRUCTIONS:")
    print("\n1. Copy the configuration above")
    print(f"\n2. Save to: {config_path}")
    print("\n3. Shutdown WSL2:")
    print("   PowerShell: wsl --shutdown")
    print("\n4. Wait 10 seconds")
    print("\n5. Restart Docker Desktop")
    print("\n6. Verify in Task Manager (Vmmem process)")
    
    print("\n" + "─" * 70)
    print("⚠️  IMPORTANT NOTES:")
    print("─" * 70)
    print(f"  • WSL2 will use UP TO {wsl_memory}, not always")
    print(f"  • Leaves {total_ram_gb - int(wsl_memory[:-2])}+ GB for Windows")
    print("  • Configuration applies to ALL WSL2 distributions")
    print("  • Changes require wsl --shutdown to take effect")
    print("  • Monitor with Task Manager after restart")
    print("=" * 70)
    
    return config_content

# Generate configs for different system sizes
for ram_size in [8, 16, 32]:
    config = generate_wslconfig(ram_size)
    print("\n" + "="*70 + "\n")

## Part 3: File System Performance

### Windows vs WSL2 File Systems

Docker on Windows can access two file systems with dramatically different performance:

| Location | Performance | Use Case |
|----------|-------------|----------|
| WSL2 native (`/home/...`) | Fast (100%) | Docker volumes, container data |
| Windows NTFS (`/mnt/c/...`) | Slow (10-20%) | Shared files with Windows apps |

**Key rule**: Docker volumes should ALWAYS use WSL2 native filesystem.

### Best Practices

#### ✅ GOOD: Use Named Volumes (WSL2 Native)

```yaml
services:
  n8n:
    volumes:
      - n8n_data:/home/node/.n8n  # Fast: WSL2 native

volumes:
  n8n_data:  # Docker manages location
```

#### ❌ BAD: Bind Mount to Windows Path

```yaml
services:
  n8n:
    volumes:
      - C:/n8n/data:/home/node/.n8n  # Slow: cross-filesystem
```

### Checking Volume Location

```powershell
# List Docker volumes
docker volume ls

# Inspect volume location
docker volume inspect n8n_data
```

### Accessing Volume Data from Windows

If you need to access n8n data from Windows:

```powershell
# 1. Find WSL2 distribution
wsl -l -v

# 2. Access via Windows Explorer
# Navigate to: \\wsl$\docker-desktop-data\version-pack-data\community\docker\volumes\n8n_data\_data
```

**Better approach**: Use `docker cp` command

```powershell
# Copy file from container to Windows
docker cp n8n:/home/node/.n8n/config C:\backup\config

# Copy from Windows to container
docker cp C:\backup\config n8n:/home/node/.n8n/config
```

### npm Installation File System

For npm installations, use SSD for best performance:

- ✅ .n8n directory on SSD (C: drive)
- ✅ Database files on SSD
- ⚠️  Large binary data can use HDD if needed

### Database Location Optimization

```batch
REM SQLite database location (npm)
SET N8N_USER_FOLDER=C:\n8n\data

REM This sets where .n8n directory is created
REM Default: C:\Users\YourUsername\.n8n
```

**Recommendation**: Keep default location unless you have specific reasons to change.

## Part 4: Battery Life Optimization (Laptops)

### Power Consumption Breakdown

| Component | Power Draw | Battery Impact |
|-----------|------------|----------------|
| n8n (npm, idle) | 2-5W | 5-10% per hour |
| n8n (npm, active) | 10-20W | 15-25% per hour |
| Docker Desktop | 5-15W | 10-20% per hour |
| WSL2 (idle) | 3-8W | 7-15% per hour |
| Total Docker stack | 8-23W | 17-35% per hour |

### Battery-Saving Strategies

#### 1. Choose npm Over Docker

```
Battery life with Docker: 2-3 hours
Battery life with npm: 4-6 hours

Savings: 50-100%
```

#### 2. Stop n8n When Not in Use

```powershell
# npm: Just close the terminal running n8n
# Or: Ctrl+C in the terminal

# Docker: Stop container
docker stop n8n

# Docker Desktop: Quit entirely
# Right-click system tray icon → Quit Docker Desktop
```

#### 3. Reduce Workflow Execution Frequency

```
Instead of: Every 5 minutes (12 per hour)
Use: Every 30 minutes (2 per hour)

Savings: 20-30% battery
```

#### 4. Disable Execution Data Saving

```batch
SET EXECUTIONS_DATA_SAVE_ON_SUCCESS=none
SET EXECUTIONS_DATA_SAVE_MANUAL_EXECUTIONS=false
```

Reduces disk writes, saving power.

#### 5. Windows Power Settings

Create a custom power plan for development:

1. Open: Control Panel → Power Options
2. Create power plan: "Development Mode"
3. Settings:
   - Turn off display: 5 minutes
   - Put computer to sleep: Never (while plugged in)
   - Put computer to sleep: 30 minutes (on battery)
   - Processor power management: 70% maximum (on battery)

#### 6. Schedule n8n for Plugged-In Times

Use Task Scheduler to auto-start only when on AC power:

```powershell
schtasks /Create /TN "n8n-on-power" /SC ONSTART /DELAY 0000:30 `
  /TR "C:\n8n\start-n8n.bat" /RL HIGHEST /F

# Then in Task Scheduler:
# Conditions → Check "Start only if computer is on AC power"
```

### Monitoring Battery Impact

```powershell
# Check battery status
powercfg /batteryreport

# Report saved to: C:\Windows\System32\battery-report.html
```

Compare battery drain with and without n8n running.

In [ ]:
# Battery optimization calculator

def calculate_battery_impact(installation_method, usage_pattern, battery_capacity_wh=50):
    """
    Calculate estimated battery impact of n8n.
    
    Args:
        installation_method: 'npm' or 'docker'
        usage_pattern: 'idle', 'light', 'moderate', 'heavy'
        battery_capacity_wh: Battery capacity in watt-hours (typical laptop: 40-60Wh)
    """
    print("=" * 70)
    print("BATTERY IMPACT CALCULATOR")
    print("=" * 70)
    
    # Power consumption estimates (watts)
    power_usage = {
        'npm': {'idle': 3, 'light': 5, 'moderate': 10, 'heavy': 15},
        'docker': {'idle': 12, 'light': 18, 'moderate': 25, 'heavy': 35}
    }
    
    if installation_method not in power_usage:
        installation_method = 'npm'
    
    watts = power_usage[installation_method].get(usage_pattern, 5)
    
    # Calculate battery drain
    hours_on_battery = battery_capacity_wh / watts
    percent_per_hour = (watts / battery_capacity_wh) * 100
    
    print(f"\n⚙️  Configuration:")
    print(f"   Installation: {installation_method.upper()}")
    print(f"   Usage pattern: {usage_pattern.upper()}")
    print(f"   Battery capacity: {battery_capacity_wh}Wh")
    
    print(f"\n🔋 Power Consumption:")
    print(f"   Estimated draw: {watts}W")
    
    print(f"\n📊 Battery Impact:")
    print(f"   Battery drain: {percent_per_hour:.1f}% per hour")
    print(f"   Runtime: {hours_on_battery:.1f} hours on battery")
    print(f"   Daily usage (8h): {percent_per_hour * 8:.0f}% battery")
    
    # Recommendations
    print("\n" + "─" * 70)
    print("💡 OPTIMIZATION RECOMMENDATIONS:")
    print("─" * 70)
    
    if installation_method == 'docker':
        npm_watts = power_usage['npm'][usage_pattern]
        savings = watts - npm_watts
        savings_percent = (savings / watts) * 100
        
        print(f"\n✅ Switch to npm installation:")
        print(f"   Power savings: {savings}W ({savings_percent:.0f}%)")
        print(f"   Extra runtime: {(battery_capacity_wh / npm_watts) - hours_on_battery:.1f} hours")
    
    if percent_per_hour > 15:
        print(f"\n⚠️  High battery drain detected ({percent_per_hour:.1f}%/hour)")
        print("\n   Consider:")
        print("   • Stop n8n when not actively developing")
        print("   • Reduce workflow execution frequency")
        print("   • Use laptop plugged in for n8n work")
    
    if usage_pattern in ['moderate', 'heavy']:
        lighter_pattern = 'light' if usage_pattern == 'moderate' else 'moderate'
        lighter_watts = power_usage[installation_method][lighter_pattern]
        print(f"\n💡 Reduce to {lighter_pattern} usage:")
        print(f"   Power savings: {watts - lighter_watts}W")
        print(f"   Extra runtime: {(battery_capacity_wh / lighter_watts) - hours_on_battery:.1f} hours")
    
    print("\n" + "=" * 70)
    print("\n📝 BATTERY-SAVING QUICK TIPS:")
    print("   1. Stop n8n when not in use (biggest impact)")
    print("   2. Use npm instead of Docker (50-70% savings)")
    print("   3. Reduce schedule trigger frequency")
    print("   4. Disable execution data saving")
    print("   5. Work plugged in for intensive workflows")
    print("=" * 70)

# Test scenarios
scenarios = [
    ('npm', 'idle', 50),
    ('npm', 'moderate', 50),
    ('docker', 'idle', 50),
    ('docker', 'moderate', 50),
]

for method, pattern, capacity in scenarios:
    calculate_battery_impact(method, pattern, capacity)
    print("\n" + "="*70 + "\n")

## Part 5: Database Choices and Configuration

### SQLite vs PostgreSQL

| Feature | SQLite | PostgreSQL |
|---------|--------|------------|
| **Setup** | Automatic | Manual installation required |
| **Performance (single user)** | Fast | Moderate |
| **Performance (concurrent)** | Poor | Excellent |
| **Resource usage** | Low (50-100MB) | Higher (200-500MB) |
| **Data integrity** | Good | Excellent |
| **Corruption risk** | Moderate | Low |
| **Backup** | Copy file | pg_dump required |
| **Best for** | Development, personal | Production, teams |

### When to Use SQLite (Default)

✅ Good for:
- Development and testing
- Personal automation (single user)
- Low-resource systems
- Simple backup requirements
- Workflows that don't run concurrently

❌ Avoid for:
- Team environments (multiple users)
- High-concurrency workflows
- Business-critical automation
- Large datasets (>100K workflow executions)

### When to Use PostgreSQL

✅ Good for:
- Production deployments
- Team environments
- High-concurrency scenarios
- Business-critical workflows
- Large execution history

❌ Overkill for:
- Personal laptop development
- Learning and testing
- Low-resource systems

### Setting Up PostgreSQL (Optional)

#### Using Docker Compose

```yaml
version: '3.8'

services:
  postgres:
    image: postgres:15
    container_name: n8n_postgres
    environment:
      - POSTGRES_USER=n8n
      - POSTGRES_PASSWORD=n8n_secure_password
      - POSTGRES_DB=n8n
    volumes:
      - postgres_data:/var/lib/postgresql/data
    restart: unless-stopped

  n8n:
    image: docker.n8n.io/n8nio/n8n
    container_name: n8n
    ports:
      - "5678:5678"
    environment:
      - DB_TYPE=postgresdb
      - DB_POSTGRESDB_HOST=postgres
      - DB_POSTGRESDB_PORT=5432
      - DB_POSTGRESDB_DATABASE=n8n
      - DB_POSTGRESDB_USER=n8n
      - DB_POSTGRESDB_PASSWORD=n8n_secure_password
      - N8N_BASIC_AUTH_ACTIVE=true
      - N8N_BASIC_AUTH_USER=admin
      - N8N_BASIC_AUTH_PASSWORD=admin_password
    volumes:
      - n8n_data:/home/node/.n8n
    depends_on:
      - postgres
    restart: unless-stopped

volumes:
  postgres_data:
  n8n_data:
```

Start with: `docker compose up -d`

### SQLite Optimization

If sticking with SQLite, optimize it:

```batch
REM Enable WAL mode for better concurrent access
SET N8N_SQLITE_VACUUM_ON_STARTUP=true

REM Regular maintenance
REM Stop n8n, then run:
sqlite3 C:\Users\YourUsername\.n8n\database.sqlite "VACUUM;"
```

## Part 6: Execution Data Pruning

### The Database Growth Problem

Without pruning:
- Active n8n instance generates 100-500MB execution data per month
- After 6 months: 600MB - 3GB database
- After 1 year: 1.2GB - 6GB database
- Queries slow down, backups take longer, disk space fills up

### Pruning Configuration

```batch
REM Enable automatic pruning
SET EXECUTIONS_DATA_PRUNE=true

REM Delete executions older than 168 hours (7 days)
SET EXECUTIONS_DATA_MAX_AGE=168

REM OR: Limit to last N executions per workflow
REM SET EXECUTIONS_DATA_SAVE_MANUAL_EXECUTIONS=false

REM Binary data storage (recommended)
SET N8N_DEFAULT_BINARY_DATA_MODE=filesystem
```

### Pruning Strategies by Use Case

#### Development/Testing

```batch
SET EXECUTIONS_DATA_SAVE_ON_SUCCESS=none
SET EXECUTIONS_DATA_SAVE_ON_ERROR=all
SET EXECUTIONS_DATA_SAVE_MANUAL_EXECUTIONS=true
SET EXECUTIONS_DATA_PRUNE=true
SET EXECUTIONS_DATA_MAX_AGE=24
```

**Result**: Only saves errors and manual tests, deletes after 1 day.

#### Personal Production

```batch
SET EXECUTIONS_DATA_SAVE_ON_SUCCESS=all
SET EXECUTIONS_DATA_SAVE_ON_ERROR=all
SET EXECUTIONS_DATA_SAVE_MANUAL_EXECUTIONS=true
SET EXECUTIONS_DATA_PRUNE=true
SET EXECUTIONS_DATA_MAX_AGE=168
SET N8N_DEFAULT_BINARY_DATA_MODE=filesystem
```

**Result**: Saves everything, keeps 7 days, binary data in files.

#### Team/Business

```batch
SET EXECUTIONS_DATA_SAVE_ON_SUCCESS=all
SET EXECUTIONS_DATA_SAVE_ON_ERROR=all
SET EXECUTIONS_DATA_SAVE_MANUAL_EXECUTIONS=true
SET EXECUTIONS_DATA_PRUNE=true
SET EXECUTIONS_DATA_MAX_AGE=720
SET N8N_DEFAULT_BINARY_DATA_MODE=filesystem
```

**Result**: Saves everything, keeps 30 days for auditing.

### Manual Pruning

If you need to manually clean up:

```powershell
# Stop n8n first

# For SQLite, connect with sqlite3
sqlite3 C:\Users\YourUsername\.n8n\database.sqlite

# Delete old executions (30+ days old)
DELETE FROM execution_entity WHERE startedAt < datetime('now', '-30 days');

# Vacuum to reclaim space
VACUUM;

# Exit
.quit
```

**Warning**: Always backup before manual database operations!

In [ ]:
# Execution data pruning calculator

def calculate_database_growth(workflows, executions_per_day, avg_execution_size_kb, retention_days):
    """
    Calculate database growth and provide pruning recommendations.
    
    Args:
        workflows: Number of active workflows
        executions_per_day: Average executions per day per workflow
        avg_execution_size_kb: Average size of execution data in KB
        retention_days: How long to keep execution data
    """
    print("=" * 70)
    print("DATABASE GROWTH CALCULATOR & PRUNING RECOMMENDATIONS")
    print("=" * 70)
    
    # Calculate growth rates
    daily_executions = workflows * executions_per_day
    daily_growth_kb = daily_executions * avg_execution_size_kb
    daily_growth_mb = daily_growth_kb / 1024
    
    monthly_growth_mb = daily_growth_mb * 30
    yearly_growth_mb = daily_growth_mb * 365
    yearly_growth_gb = yearly_growth_mb / 1024
    
    # With pruning
    steady_state_kb = daily_executions * retention_days * avg_execution_size_kb
    steady_state_mb = steady_state_kb / 1024
    steady_state_gb = steady_state_mb / 1024
    
    print(f"\n📊 CONFIGURATION:")
    print(f"   Active workflows: {workflows}")
    print(f"   Executions per day: {executions_per_day} per workflow")
    print(f"   Total daily executions: {daily_executions}")
    print(f"   Average execution size: {avg_execution_size_kb} KB")
    
    print(f"\n📈 WITHOUT PRUNING (growth rate):")
    print(f"   Per day: {daily_growth_mb:.1f} MB")
    print(f"   Per month: {monthly_growth_mb:.0f} MB")
    print(f"   Per year: {yearly_growth_gb:.2f} GB")
    
    print(f"\n✂️  WITH PRUNING ({retention_days} day retention):")
    print(f"   Steady-state size: {steady_state_mb:.0f} MB ({steady_state_gb:.2f} GB)")
    print(f"   Space saved per year: {yearly_growth_gb - steady_state_gb:.2f} GB")
    
    # Recommendations
    print("\n" + "─" * 70)
    print("💡 RECOMMENDATIONS:")
    print("─" * 70)
    
    if yearly_growth_gb > 5:
        print("\n⚠️  HIGH DATABASE GROWTH ({yearly_growth_gb:.1f} GB/year)")
        print("\n   CRITICAL: Enable pruning immediately")
        print("\n   Recommended settings:")
        print("   SET EXECUTIONS_DATA_PRUNE=true")
        print("   SET EXECUTIONS_DATA_MAX_AGE=168  # 7 days")
        print("   SET N8N_DEFAULT_BINARY_DATA_MODE=filesystem")
        
        if avg_execution_size_kb > 100:
            print("\n   ⚠️  Large execution data detected")
            print("   • Enable binary data filesystem storage")
            print("   • Reduce data passed between nodes")
            print("   • Consider saving only errors")
    
    elif yearly_growth_gb > 1:
        print("\n✅ MODERATE DATABASE GROWTH ({yearly_growth_gb:.1f} GB/year)")
        print("\n   Recommended settings:")
        print("   SET EXECUTIONS_DATA_PRUNE=true")
        print(f"   SET EXECUTIONS_DATA_MAX_AGE={min(retention_days, 168)}")
        print("   SET N8N_DEFAULT_BINARY_DATA_MODE=filesystem")
    
    else:
        print("\n✅ LOW DATABASE GROWTH ({yearly_growth_gb:.1f} GB/year)")
        print("\n   Current growth is manageable")
        print("   Optional: Enable pruning for long-term stability")
        print("   SET EXECUTIONS_DATA_PRUNE=true")
        print("   SET EXECUTIONS_DATA_MAX_AGE=720  # 30 days")
    
    # Optimal retention calculation
    print("\n" + "─" * 70)
    print("📅 RETENTION PERIOD GUIDE:")
    print("─" * 70)
    
    retention_options = {
        '24 hours (1 day)': 24,
        '3 days': 72,
        '7 days': 168,
        '14 days': 336,
        '30 days': 720,
        '90 days': 2160
    }
    
    print("\n   Period          DB Size    Use Case")
    print("   " + "─" * 60)
    
    for label, hours in retention_options.items():
        days = hours / 24
        size_kb = daily_executions * days * avg_execution_size_kb
        size_mb = size_kb / 1024
        
        if size_mb < 100:
            size_str = f"{size_mb:.0f} MB"
        else:
            size_str = f"{size_mb/1024:.1f} GB"
        
        if hours == 24:
            use_case = "Development/testing"
        elif hours <= 168:
            use_case = "Personal production"
        elif hours <= 720:
            use_case = "Team/business"
        else:
            use_case = "Compliance/audit"
        
        print(f"   {label:<15} {size_str:<10} {use_case}")
    
    print("\n" + "=" * 70)

# Test scenarios
scenarios = [
    (5, 10, 50, 168),  # 5 workflows, 10 runs/day, 50KB each, 7 day retention
    (20, 50, 100, 168),  # Moderate usage
    (50, 100, 200, 168),  # Heavy usage
]

for workflows, exec_per_day, size_kb, retention in scenarios:
    calculate_database_growth(workflows, exec_per_day, size_kb, retention)
    print("\n" + "="*70 + "\n")

## Exercises

### Exercise 1: Configure WSL2 Memory Limits (Easy)

**Task**: Create a .wslconfig file to limit WSL2 memory usage.

**Steps**:
1. Determine your system RAM (Task Manager → Performance → Memory)
2. Use the .wslconfig generator above to create configuration
3. Save configuration to `C:\Users\YourUsername\.wslconfig`
4. Shutdown WSL2: `wsl --shutdown`
5. Wait 10 seconds
6. Restart Docker Desktop
7. Monitor vmmem process in Task Manager
8. Verify memory usage stays under configured limit

**Verification**: vmmem memory usage should not exceed your configured limit.

**Deliverable**: Screenshot of Task Manager showing vmmem under limit.

### Exercise 2: Implement Execution Data Pruning (Medium)

**Task**: Configure automatic pruning to prevent database bloat.

**Steps**:
1. Stop n8n if running
2. Check current database size (`.n8n/database.sqlite`)
3. Update your startup script with:
   - `EXECUTIONS_DATA_PRUNE=true`
   - `EXECUTIONS_DATA_MAX_AGE=168` (7 days)
   - `N8N_DEFAULT_BINARY_DATA_MODE=filesystem`
   - `EXECUTIONS_DATA_SAVE_ON_SUCCESS=all` (or `none` for testing)
4. Restart n8n
5. Create and run a test workflow multiple times
6. Check execution history (should show your runs)
7. Wait 24 hours and verify old executions are pruned

**Verification**: Execution history shows only recent runs, database size is stable.

**Deliverable**: Configuration file and execution history screenshot.

### Exercise 3: Performance Benchmark (Hard)

**Task**: Compare performance of npm vs Docker installation on your system.

**Steps**:
1. If using Docker, note current memory usage (Task Manager)
2. Stop Docker n8n
3. Install npm n8n (if not already installed)
4. Configure npm n8n with same settings
5. Import your workflows
6. Measure:
   - Idle memory usage
   - Startup time
   - Workflow execution speed (create identical test workflow)
   - Battery impact (if on laptop) - run for 30 minutes
7. Document results
8. Calculate which method is better for your use case

**Verification**: Complete comparison table with measured data.

**Deliverable**: Performance comparison report with recommendation for your use case.

## Summary

In this module, you learned comprehensive performance optimization for n8n on Windows:

### Memory Management
1. **npm vs Docker**: 20-40x memory difference at idle
2. **Node.js limits**: Use `NODE_OPTIONS=--max-old-space-size=1024`
3. **Container limits**: Configure memory limits in Docker Compose
4. **Execution data**: Save only what you need to reduce memory

### WSL2 Optimization
5. **.wslconfig file**: Essential for Docker Desktop users
6. **Memory limits**: Prevent WSL2 from consuming 50%+ of RAM
7. **Processor limits**: Balance performance with other Windows apps
8. **Swap configuration**: Virtual memory for handling peaks

### File System Performance
9. **WSL2 native vs NTFS**: 10x performance difference
10. **Named volumes**: Always use for Docker data
11. **SSD placement**: Keep databases on SSD for best performance
12. **Access patterns**: Use `docker cp` for file transfers

### Battery Life Optimization
13. **Installation choice**: npm uses 50-70% less power than Docker
14. **Stop when idle**: Biggest battery savings
15. **Reduce frequency**: Schedule triggers less often
16. **Power plans**: Windows settings for laptop users

### Database Configuration
17. **SQLite**: Best for development and personal use
18. **PostgreSQL**: Better for production and teams
19. **Performance trade-offs**: Resource usage vs data integrity
20. **Migration strategy**: When to switch databases

### Execution Data Pruning
21. **Automatic pruning**: `EXECUTIONS_DATA_PRUNE=true`
22. **Retention periods**: 24 hours to 90 days based on use case
23. **Binary data**: Store in filesystem, not database
24. **Database growth**: Prevent multi-GB database sizes

### Key Takeaways

✅ **Choose npm for laptops** - 50-70% better battery life and memory  
✅ **Configure .wslconfig** - Essential for Docker Desktop  
✅ **Enable pruning** - Prevents database bloat  
✅ **Use named volumes** - 10x faster than bind mounts  
✅ **Monitor resources** - Task Manager is your friend  
✅ **Stop when idle** - Saves battery and resources  

### Performance Optimization Matrix

| Use Case | Installation | Memory Config | Database | Pruning |
|----------|--------------|---------------|----------|----------|
| Development | npm | 512-1024 MB | SQLite | 24-72 hours |
| Personal Production | npm | 1024-2048 MB | SQLite | 7 days |
| Team | Docker | 2048-4096 MB | PostgreSQL | 30 days |
| Enterprise | Docker/Server | 4096+ MB | PostgreSQL | 90 days |

### What's Next

- **Module 08**: Automatic startup configuration (Task Scheduler, NSSM, PM2)
- **Module 09**: Updates and maintenance procedures
- **Module 10**: Production deployment strategies

### Additional Resources

- [WSL2 Best Practices](https://docs.microsoft.com/en-us/windows/wsl/wsl-config)
- [Docker Desktop Performance](https://docs.docker.com/desktop/windows/wsl/)
- [n8n Performance Tips](https://docs.n8n.io/hosting/configuration/performance/)
- [SQLite Optimization](https://www.sqlite.org/optoverview.html)

### Remember

Performance optimization is about finding the right balance for YOUR use case. Development on battery? Choose npm and aggressive pruning. Team production? Docker with PostgreSQL makes sense. Monitor your actual usage and adjust these recommendations based on real-world performance.